In [ ]:
import os 

from random import random, randrange
from xmlrpc.client import MAXINT
from bs4 import BeautifulSoup
import numpy as np

import time
import requests
import re
import codecs
import pandas as pd
import concurrent.futures
import matplotlib.pyplot as plt


N_THREADS = 4
ZEROS_STRING = "".join("0" for i in range(7))

url = 'http://www.imdb.com/title/tt'

APPEND_LIMIT = 100

data = []

In [ ]:
# Data collection from the previous work
movies = pd.read_parquet("Titles.parquet")
movies = movies[movies["titleType"] == "movie"].reset_index(drop = True)

In [ ]:
ids = movies.id.values

In [ ]:
os.mkdir("C:\\Users\\a.atmaeva\\img")

In [ ]:
#Add description and pictures
def scrap_titles(ids):

    with concurrent.futures.ThreadPoolExecutor(max_workers=N_THREADS) as executor:
        executor.map(scrap_title_by_id, ids)


def scrap_title_by_id(id):
    title = origin_title = age_restriction = country_of_origin = duration = director = release_date = budget = gorss_worldwide = rating = number_of_ratings = user_reveiws = crtitic_reviews = None
    genres = []
    if len(str(id)) == 8:
        REVIEW_ID_LENGTH = 8
    else:
        REVIEW_ID_LENGTH = 7
    response = requests.get(
        url + ZEROS_STRING[:REVIEW_ID_LENGTH - len(str(id))] + str(id))
    print(
        f'{url + ZEROS_STRING[:REVIEW_ID_LENGTH - len(str(id))] + str(id)}    {response.status_code}')
    if response.status_code == 503: 
            time.sleep(1200)
    soup = BeautifulSoup(response.text, 'lxml')

    title = soup.find(
        "h1", {"data-testid": "hero-title-block__title"})
    if title:
        title = title.text

    origin_title = soup.find(
        "div", {"data-testid": "hero-title-block__original-title"})
    if origin_title:
        origin_title = origin_title.text.split(": ")[1]

    
    film_description = soup.find("span", {"data-testid":"plot-xl"}).text

    image = soup.find('img')
    image_url = image['src']
    r = requests.get(image_url, stream = True).content
    with open(f"img/{id}.jpg", "wb+") as f:
                        f.write(r)
    print(str(id) + " " + str(title) + " " + str(origin_title)  + str(film_description))

    print(f'Data length: {len(data)}')
    
    data.append((id, title, origin_title, film_description))
    if len(data) == APPEND_LIMIT:
        df = pd.DataFrame(data)
        df.to_csv('Titles_updated.csv', mode = 'a', index = False, header = False)
        data.clear()

scrap_titles(ids)